<a href="https://colab.research.google.com/github/yashmathuria0001/Stock-Market-Agent/blob/main/stockagent_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install langchain langchain-community groq yfinance langchain_groq tavily-python --quiet

In [42]:
!pip install -qU duckduckgo-search

In [43]:
!pip install -U langchain-google-genai google-generativeai langchain yfinance langchain-experimental --quiet


In [44]:
from google.colab import userdata
# from langchain_groq import ChatGroq
import google.generativeai as genai

gemini_api = userdata.get('gemini_api')
genai.configure(api_key=gemini_api)

# Initialize Gemini model
llm = genai.GenerativeModel("gemini-2.0-flash")



# llm = ChatGroq(model_name="llama3-70b-8192", api_key=userdata.get('new_api'))

In [48]:
import os
import yfinance as yf
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import GoogleGenerativeAI  # ✅ Correct Import
import ast
import operator as op
from google.colab import userdata

# ✅ Get Gemini API Key from Colab Secure Storage
gemini_api = userdata.get('gemini_api')

# ✅ Initialize Gemini AI in LangChain
llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_api)

# ✅ Fetch Stock Price from Yahoo Finance
def get_stock_price(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        price = stock.history(period="1d")["Close"].iloc[-1]
        return f"The latest stock price of {ticker} is ${price:.2f}"
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

# ✅ Fetch Financial Report from Yahoo Finance
def get_finance_report(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        financials = stock.financials
        report = f"Company: {info.get('longName', ticker)}\n"
        report += f"Sector: {info.get('sector', 'N/A')}\n"
        report += "Recent Financials:\n"
        report += str(financials.head())
        return report
    except Exception as e:
        return f"Error fetching finance report for {ticker}: {str(e)}"

# ✅ Enhanced Stock Analysis for Buy/Sell/Hold Recommendation using Gemini AI
def analyze_stock(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        history = stock.history(period="1y")  # Last 1-year data for better accuracy

        if history.empty:
            return f"Could not fetch data for {ticker}. Try again later."

        latest_price = history["Close"].iloc[-1]
        last_week_price = history["Close"].iloc[-5] if len(history) > 5 else history["Close"].iloc[0]
        last_month_price = history["Close"].iloc[-22] if len(history) > 22 else history["Close"].iloc[0]
        last_year_price = history["Close"].iloc[0]

        weekly_change = ((latest_price - last_week_price) / last_week_price) * 100
        monthly_change = ((latest_price - last_month_price) / last_month_price) * 100
        yearly_change = ((latest_price - last_year_price) / last_year_price) * 100

        prompt = f"""
        Analyze the stock performance for {ticker} based on historical price movements and market trends.

        Key Data:
        - Latest Price: ${latest_price:.2f}
        - Weekly Change: {weekly_change:.2f}%
        - Monthly Change: {monthly_change:.2f}%
        - Yearly Change: {yearly_change:.2f}%

        Provide a recommendation (Buy, Sell, Hold) considering:
        - Short-term and long-term trends
        - Market volatility and investor sentiment
        - Potential risks and opportunities
        - Justify your recommendation with clear reasoning and insights.
        """

        response = llm.invoke(prompt)
        return response
    except Exception as e:
        return f"Error analyzing stock {ticker}: {str(e)}"

# ✅ Stop-Loss Price Calculation using Gemini AI
def get_stop_loss_price(ticker: str, entry_price: float, risk_percent: float = 2.0) -> str:
    try:
        stock = yf.Ticker(ticker)
        latest_price = stock.history(period="1d")["Close"].iloc[-1]

        stop_loss_price = entry_price * (1 - risk_percent / 100)

        prompt = f"""
        Calculate an appropriate stop-loss price for {ticker} based on the following:
        - Entry Price: ${entry_price:.2f}
        - Latest Market Price: ${latest_price:.2f}
        - Risk Percentage: {risk_percent:.2f}%

        Consider:
        - Market volatility and recent fluctuations
        - Support and resistance levels
        - Optimal risk management strategies

        Provide a recommendation on where to set the stop-loss and justify the decision.
        """

        response = llm.invoke(prompt)
        return response
    except Exception as e:
        return f"Error calculating stop-loss for {ticker}: {str(e)}"

# ✅ Create LangChain Tools
finance_report_tool = Tool(
    name="Yahoo Finance Report",
    func=get_finance_report,
    description="Fetches key financial data for a given ticker symbol (e.g., AAPL, TSLA, MSFT)."
)

stock_price_tool = Tool(
    name="Stock Price Fetcher",
    func=get_stock_price,
    description="Fetches the latest stock price for a given ticker symbol (e.g., AAPL, TSLA, MSFT)."
)

stock_analysis_tool = Tool(
    name="Stock Analysis Tool",
    func=analyze_stock,
    description="Analyzes stock trends over 1 year and recommends Buy, Sell, or Hold using Gemini AI. Example: 'AAPL'."
)

stop_loss_tool = Tool(
    name="Stop Loss Calculator",
    func=lambda x: get_stop_loss_price(x.split(',')[0], float(x.split(',')[1]), float(x.split(',')[2]) if len(x.split(',')) > 2 else 2.0),
    description="Calculates an appropriate stop-loss price based on entry price and risk percentage. Example: 'AAPL,230,1.5' or 'AAPL,230' (default 2% risk)."
)

# ✅ Initialize LangChain Agent with Gemini AI
agent = initialize_agent(
    tools=[stock_price_tool, finance_report_tool, stock_analysis_tool, stop_loss_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [49]:
response = agent.invoke("give recommandation for microsoft")
print(response)




> Entering new AgentExecutor chain...
I need to analyze the stock trends for Microsoft to provide a recommendation.
Action: Stock Analysis Tool
Action Input: MSFT
Observation: Okay, let's analyze Microsoft's (MSFT) stock performance based on the provided data and current market conditions.

**Analysis of Microsoft (MSFT) Stock Performance**

*   **Latest Price:** $393.08
*   **Weekly Change:** 2.49% (Positive)
*   **Monthly Change:** -3.71% (Negative)
*   **Yearly Change:** -6.33% (Negative)

**Interpretation of Data:**

*   **Short-Term (Weekly):** The positive weekly change suggests a recent upward trend.  This could be due to a short-term catalyst, positive news, or simply a correction after a previous dip.
*   **Mid-Term (Monthly):** The negative monthly change indicates some weakness in the past month. This suggests the recent positive week might be a counter-trend rally or a recovery from a more significant decline.
*   **Long-Term (Yearly):** The negative yearly change paints 